In [54]:
import pandas as pd
import numpy as np
import pylab as pl

from sklearn import linear_model
from sklearn.linear_model import LogisticRegression, RidgeClassifierCV
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.cross_validation import cross_val_score
from sklearn.preprocessing import Imputer
from sklearn import preprocessing
from sklearn.base import BaseEstimator
from sklearn.cross_validation import train_test_split
from sklearn import metrics
from sklearn.cross_validation import cross_val_score

import os
import json

%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
a = pd.read_csv('Theory.csv')

In [3]:
a.head()

,ID_INTERNAL,CD_TYPE_ACCT,ID_ACCT,DT_PD_ACCT_BEGIN,DT_PD_ACCT_END,QT_ASSESSMENT,CD_TYPE_ASSESSMENT,AM_LIAB_CANCEL,AM_COLL_CANCEL,DT_NOTICE_DEM_PYMT,...,BalanceGZero,Warranted,AM_TAX_DUE,AM_PEN_UNDERPYMT,AM_PEN_LATE_FILE,AM_PEN_LATE_PYMT,AM_ABT_UNDERPYMT,AM_ABT_LATE_FILE,AM_ABT_LATE_PYMT,AM_TAX_MATH_ERROR
0,30034666303,200,NaN,2013-11-01,2014-09-29,1,1,0.00,0,2014-12-24,...,0,1,25.00,0.00,0,0.00,0,0,0,0.00
1,30030541103,200,NaN,2013-01-01,2013-12-31,1,1,0.00,0,9999-12-31,...,0,0,9994.00,315.98,0,0.00,0,0,0,-0.14
2,30030384403,200,NaN,2013-05-01,2014-04-30,1,1,-2.13,0,9999-12-31,...,0,0,241.00,0.00,0,0.00,0,0,0,0.25
3,30036523603,200,NaN,2013-10-17,2013-12-31,2,2,0.00,0,9999-12-31,...,0,0,2212.65,0.00,0,32.86,0,0,0,-0.15
4,30028953903,200,NaN,2013-10-01,2014-09-30,1,1,0.00,0,9999-12-31,...,0,0,32.00,0.00,0,0.00,0,0,0,-0.41


In [6]:
a.describe()

,ID_INTERNAL,CD_TYPE_ACCT,ID_ACCT,QT_ASSESSMENT,CD_TYPE_ASSESSMENT,AM_LIAB_CANCEL,AM_COLL_CANCEL,AM_LIABILITY,AM_COLLECTIONS,AM_ASMT_BALANCE,...,BalanceGZero,Warranted,AM_TAX_DUE,AM_PEN_UNDERPYMT,AM_PEN_LATE_FILE,AM_PEN_LATE_PYMT,AM_ABT_UNDERPYMT,AM_ABT_LATE_FILE,AM_ABT_LATE_PYMT,AM_TAX_MATH_ERROR
count,6.553500e+04,65535.000000,0,65535.000000,65535.000000,65535.000000,65535,65535.000000,65535.000000,65535.000000,...,65535.000000,65535.000000,65535.000000,65535.000000,65535.000000,65535.000000,65535.000000,65535.000000,65535.000000,65535.000000
mean,2.244230e+10,244.563210,NaN,1.006592,1.013077,-0.521380,0,10978.471481,10627.308525,351.162956,...,0.071397,0.027451,10753.559012,50.523602,120.787916,24.430680,-7.199800,-11.538842,-1.607595,0.288829
std,2.031230e+10,61.050942,NaN,0.094655,0.211655,66.821971,0,239495.549364,221985.935689,95067.777420,...,0.257489,0.163395,230732.458481,2980.940096,15597.849120,2823.348227,476.588065,912.359157,170.852868,89.054678
min,2.800000e+03,10.000000,NaN,1.000000,1.000000,-10858.570000,0,-226903.540000,-226903.540000,-2445558.810000,...,0.000000,0.000000,-226904.000000,-0.010000,0.000000,0.000000,-74173.900000,-151034.100000,-30674.060000,-2878.830000
25%,1.000945e+10,200.000000,NaN,1.000000,1.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2.001174e+10,200.000000,NaN,1.000000,1.000000,0.000000,0,27.670000,25.000000,0.000000,...,0.000000,0.000000,25.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,3.001356e+10,320.000000,NaN,1.000000,1.000000,0.000000,0,1021.690000,990.345000,0.000000,...,0.000000,0.000000,984.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
max,7.001227e+10,330.000000,NaN,5.000000,7.000000,0.000000,0,23861041.110000,23263263.460000,23861041.110000,...,1.000000,1.000000,23249130.000000,739088.350000,3952641.110000,702691.750000,0.000000,0.000000,0.000000,22610.720000


In [8]:
a.dtypes

ID_INTERNAL             int64
CD_TYPE_ACCT            int64
ID_ACCT               float64
DT_PD_ACCT_BEGIN       object
DT_PD_ACCT_END         object
QT_ASSESSMENT           int64
CD_TYPE_ASSESSMENT      int64
AM_LIAB_CANCEL        float64
AM_COLL_CANCEL          int64
DT_NOTICE_DEM_PYMT     object
DT_WARRANT             object
AM_LIABILITY          float64
AM_COLLECTIONS        float64
AM_ASMT_BALANCE       float64
AM_PYMT_TOTAL         float64
YearBegin               int64
BalanceGZero            int64
Warranted               int64
AM_TAX_DUE            float64
AM_PEN_UNDERPYMT      float64
AM_PEN_LATE_FILE      float64
AM_PEN_LATE_PYMT      float64
AM_ABT_UNDERPYMT      float64
AM_ABT_LATE_FILE      float64
AM_ABT_LATE_PYMT      float64
AM_TAX_MATH_ERROR     float64
dtype: object

In [9]:
# Create unique receivable id
# I can create the key while converting int values to string (object) values
a['RecKey'] = a.ID_INTERNAL.map(str) + a.CD_TYPE_ACCT.map(str) + a.ID_ACCT + a.QT_ASSESSMENT.map(str) + a.DT_PD_ACCT_BEGIN

In [ ]:
# I want to perform a supervised perdiction of a measurement/outcome to
# accurately predict unseen test cases
# understand which inputs affect the outcome, 
# asses the quality of predictions and inferences

# Features I want to use: 
# Tax Type (GCT, UBT, Bank)
# Filing period
# Amount Due - categories up to 5k, 5 to 50K, 50 to 100k, 100 to 500k (match gct min tax categories)
# Assessment Type 
# Penalties
# Abatements
# Remaining Balance greater 0
   


In [17]:
# account type feature
actdummy = pd.get_dummies(a['CD_TYPE_ACCT'])
actdummy.columns = ['Bank', 'GCT', 'UBTI','UBTP']

In [18]:
actdummy.dtypes

Bank    float64
GCT     float64
UBTI    float64
UBTP    float64
dtype: object

In [19]:
a = pd.concat([a, actdummy], axis=1)

In [20]:
#Period feature
yeardummy = pd.get_dummies(a['YearBegin'])

In [21]:
a = pd.concat([a, yeardummy], axis=1)

In [31]:
#amount feature
a['500KOver'] = a["AM_LIABILITY"].map(lambda x: 1 if x >= 500000 else 0)
a['100to500K'] = a["AM_LIABILITY"].map(lambda x: 1 if 100000 < x <= 500000 else 0)
a['50to100K'] = a["AM_LIABILITY"].map(lambda x: 1 if 50000 < x <= 100000 else 0)
a['5to50k'] = a["AM_LIABILITY"].map(lambda x: 1 if 5000 < x <= 50000 else 0)
a['5kUnder'] = a["AM_LIABILITY"].map(lambda x: 1 if x <= 5000 else 0)

In [28]:
# Assessment Type Feature
asmtdummy = pd.get_dummies(a['CD_TYPE_ASSESSMENT'])

In [29]:
a = pd.concat([a, asmtdummy], axis=1)

In [32]:
#Late File penalty
a['LFPen'] = a["AM_PEN_LATE_FILE"].map(lambda x: 1 if x > 0 else 0)

In [33]:
#Underpayment penalty
a['UPen'] = a["AM_PEN_UNDERPYMT"].map(lambda x: 1 if x > 0 else 0)

In [34]:
#Late Pay penalty
a['LPPen'] = a["AM_PEN_LATE_PYMT"].map(lambda x: 1 if x > 0 else 0)

In [35]:
#Late Pay abatement
a['LPAbat'] = a["AM_ABT_LATE_PYMT"].map(lambda x: 1 if x > 0 else 0)

In [36]:
#Late File abatement
a['LFAbat'] = a["AM_ABT_LATE_FILE"].map(lambda x: 1 if x > 0 else 0)

In [37]:
#Underpayment abatement
a['UPAbat'] = a["AM_ABT_UNDERPYMT"].map(lambda x: 1 if x > 0 else 0)

In [38]:
#Math Error Feature
a['MathError'] = a["AM_TAX_MATH_ERROR"].map(lambda x: 1 if x > 0 else 0)

In [39]:
#payments
a['Payments'] = a["AM_PYMT_TOTAL"].map(lambda x: 1 if x > 0 else 0)

Prepare Data for Logistic Regression

In [41]:
#drop the columns i don't need
a.dtypes

ID_INTERNAL             int64
CD_TYPE_ACCT            int64
ID_ACCT               float64
DT_PD_ACCT_BEGIN       object
DT_PD_ACCT_END         object
QT_ASSESSMENT           int64
CD_TYPE_ASSESSMENT      int64
AM_LIAB_CANCEL        float64
AM_COLL_CANCEL          int64
DT_NOTICE_DEM_PYMT     object
DT_WARRANT             object
AM_LIABILITY          float64
AM_COLLECTIONS        float64
AM_ASMT_BALANCE       float64
AM_PYMT_TOTAL         float64
YearBegin               int64
BalanceGZero            int64
Warranted               int64
AM_TAX_DUE            float64
AM_PEN_UNDERPYMT      float64
AM_PEN_LATE_FILE      float64
AM_PEN_LATE_PYMT      float64
AM_ABT_UNDERPYMT      float64
AM_ABT_LATE_FILE      float64
AM_ABT_LATE_PYMT      float64
AM_TAX_MATH_ERROR     float64
RecKey                 object
Bank                  float64
GCT                   float64
UBTI                  float64
UBTP                  float64
2012                  float64
2013                  float64
2014      

In [ ]:
del a [ID_INTERNAL]
del a [CD_TYPE_ACCT]

In [46]:
X = a[['Bank','GCT','UBTI', 'UBTP',2012,2013,2014,'MathError',1,2,3,4,5,7,'500KOver','100to500K','50to100K','5to50k','5kUnder','LFPen','UPen','LPPen','LPAbat','LFAbat','UPAbat','Payments']]

In [47]:
y = a[['Warranted']]

In [49]:
# flatten y into a 1-D array
y = np.ravel(y)

In [50]:
# instantiate a logistic regression model, and fit with X and y
model = LogisticRegression()
model = model.fit(X, y)

# check the accuracy on the training set
model.score(X, y)

0.9760128175783932

In [51]:
# what percentage had warrants?
y.mean()

0.027450980392156862

In [52]:
# examine the coefficients
pd.DataFrame(zip(X.columns, np.transpose(model.coef_)))

,0,1
0,Bank,[0.998463989811]
1,GCT,[0.52095504491]
2,UBTI,[-0.594689082661]
3,UBTP,[-3.63306735308]
4,2012,[-0.530474551745]
5,2013,[-0.600821711347]
6,2014,[-1.57704113781]
7,MathError,[0.000542667197284]
8,1,[-0.39057423921]
9,2,[-1.61025374209]


In [55]:
# evaluate the model by splitting into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)
model2 = LogisticRegression()
model2.fit(X_train, y_train)

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, penalty='l2', random_state=None, tol=0.0001)

In [56]:
# predict class labels for the test set
predicted = model2.predict(X_test)
print predicted

[0 0 0 ..., 0 0 0]


In [57]:
# generate class probabilities
probs = model2.predict_proba(X_test)
print probs

[[  9.93199305e-01   6.80069507e-03]
 [  9.92496337e-01   7.50366271e-03]
 [  9.97458868e-01   2.54113216e-03]
 ..., 
 [  8.51856155e-01   1.48143845e-01]
 [  9.96924629e-01   3.07537103e-03]
 [  9.99611864e-01   3.88135696e-04]]


In [58]:
# generate evaluation metrics
print metrics.accuracy_score(y_test, predicted)
print metrics.roc_auc_score(y_test, probs[:, 1])

0.976196531204
0.940029668067


In [59]:
print metrics.confusion_matrix(y_test, predicted)
print metrics.classification_report(y_test, predicted)

[[19085    62]
 [  406   108]]
             precision    recall  f1-score   support

          0       0.98      1.00      0.99     19147
          1       0.64      0.21      0.32       514

avg / total       0.97      0.98      0.97     19661



In [60]:
# evaluate the model using 10-fold cross-validation
scores = cross_val_score(LogisticRegression(), X, y, scoring='accuracy', cv=10)
print scores
print scores.mean()

[ 0.97238328  0.9722307   0.97482453  0.97604516  0.97574001  0.97604516
  0.97817793  0.97848314  0.97680452  0.97588523]
0.975661965762
